In [1]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import base64

# Using Pixtral for image analysis

Pixtral is Mistrral's [free model](https://docs.mistral.ai/getting-started/models/models_overview/) with image understanding capabilities.

This model allows to send an image via an URL, with a written message to ask a question about an image. The model responds with a description of that image.

For this small example, we'll use [lorem-picsum](https://picsum.photos/images) images. 

## Analyzing images from an ULR

We create a `load_system_prompt` function to create an optional `SystemPrompt` LangChain object. 

We alse create a `analyze_image` function. Here, notice how the `content` argument is now a dictionary with a `text` and an `image_url` attributes:

```
{"type": "text", "text": "Please describe the scene in this image:"},
{"type": "image_url", "image_url": uri_or_url},
```


In [2]:
def load_system_prompt(filename):
    with open(filename, "r", encoding="utf-8") as file:
        content = file.read()
    return SystemMessage(content=content)

In [3]:
def analyze_image(
    uri_or_url,
    system_prompt=None,
    text="Please describe the scene in this image:",
    model_name="pixtral-12b-2409",
    temperature=0.6,
):
    chat_bot = ChatMistralAI(model=model_name, temperature=temperature)
    message_history = []

    if system_prompt:
        message_history.append(system_prompt)

    message = HumanMessage(
        content=[
            {"type": "text", "text": text},
            {"type": "image_url", "image_url": uri_or_url},
        ]
    )
    message_history.append(message)

    return chat_bot.invoke(message_history)

In [4]:
system_prompt = load_system_prompt("./system_prompt.txt")

In [5]:
image_url = "https://fastly.picsum.photos/id/17/2500/1667.jpg?hmac=HD-JrnNUZjFiP2UZQvWcKrgLoC_pc_ouUSWv8kHsJJY"

In [11]:
analyze_image(image_url, system_prompt=system_prompt)

AIMessage(content='The image depicts an ancient stone church set against a backdrop of a cloudy sky. The church is constructed from large, irregularly shaped stones, giving it a rustic and historical appearance. The roof is made of dark slate tiles, contrasting with the lighter stone walls.\n\nThe church has a prominent square tower on its left side, topped with a small, arched window. Below the tower, there is a larger arched doorway, which serves as the main entrance. The door is made of dark wood and features a simple, arched design.\n\nOn the front facade of the church, there are three arched windows, each with a simple stone frame. The windows are small and rectangular, allowing limited light into the interior of the building. The walls of the church show signs of age and weathering, with some areas of the stonework appearing worn or eroded.\n\nSurrounding the church is a well-maintained grassy area, dotted with wildflowers. In the background, there are rolling hills and a few sca

## Sending a local image

It's not possible to send a local image directly to the model, but we can convert the image into a base64-encoded data URI and send that instead. This allows the model to receive the image as if it were hosted online.

We create a helper function create_uri to handle this conversion:

In [7]:
def create_uri(image_path):
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
    return f"data:image/jpeg;base64,{encoded_image}"

Once encoded, we can pass the resulting URI to the analyze_image function just like a regular URL. The model will process and describe the image as usual.

In [8]:
impage_uri = create_uri("./lorem_picsum.jpg")

In [13]:
response = analyze_image(impage_uri, system_prompt=system_prompt)

In [14]:
response.content

'The image depicts a serene and tranquil natural scene. A narrow, winding gravel path, composed of small gray stones, cuts through a lush green field. The path is flanked by vibrant green grass on both sides, with patches of taller, slightly yellowed grass near the edges. In the distance, a line of trees, predominantly coniferous with dense, dark green foliage, forms a natural backdrop. Among these trees, there are a few deciduous trees with white blossoms, adding a touch of contrast and softness to the scene. The sky above is a clear, bright blue, suggesting a sunny day. The overall mood of the image is peaceful and inviting, evoking a sense of calm and solitude.'